In [1]:
from torch import cuda, bfloat16
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
import pandas as pd
import os

model_id = 'meta-llama/Llama-2-13b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'


hf_token = os.environ.get('hf_token')

# Need auth token for these
hf_auth = hf_token
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded on cuda:0


In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

In [3]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Use PyBGPStream to perform BGP analysis and detect anomalies in AS path lengths for IPv4 prefixes over two time periods: from January 15, 2020, 15:00 to January 15, 2020, 17:00, and January 18, 2020, 12:00 to January 18, 2020, 13:00."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1012)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Use PyBGPStream to perform BGP analysis and detect anomalies in AS path lengths for IPv4 prefixes over two time periods: from January 15, 2020, 15:00 to January 15, 2020, 17:00, and January 18, 2020, 12:00 to January 18, 2020, 13:00. [/INST]  Sure, here's an example of how you could use PyBGPStream to perform BGP analysis and detect anomalies in AS path lengths for IPv4 prefixes over two time periods:

First, you would need to install PyBGPStream using pip:
```
pip install pbgpstream
```
Next, you would need to download the BGP data for the two time periods you are interested in. You can do this using the `pbgpstream.download_bgp_data()` function, which takes the following arguments:

* `start_time`: the start time of the first time period (in ISO 8601 format)
* `end_time`: the end time of the second time period (in ISO 8601 format)
* `prefix`: the IPv4 prefix you are interested in (in CIDR notation)
* `as_set`: the set of AS numbers you are interested in (as a list of intege

In [3]:
import json
input_file_path = "/home/hb/LLM-research/evaluation/bgp_test_QA_val.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)

generated_answers_qa = []

n_question = 0
for q_dict in questions_data:
    n_question += 1
    question = q_dict["question"]
    # Generate answer using the language model
    prompt = f"[INST]{question}[/INST]"
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Output: {generated_answer}")
    generated_answers_qa.append(generated_answer)
    print(f"---------------------{n_question}-----------------------------")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: [INST]What is the purpose of BGP Update messages, and how do they carry route information?[/INST]  Border Gateway Protocol (BGP) Update messages are used to exchange route information between BGP peers. The purpose of BGP Update messages is to convey information about the state of the network, such as changes in the reachability of networks or the availability of new paths.

There are two types of BGP Update messages:

1. Open Message: This message is sent when a BGP peer is first established, and it contains information about the sender's AS number, the IP address of the BGP speaker, and other parameters.
2. Update Message: This message is sent when there is a change in the network, such as a change in the reachability of a network or the availability of a new path. The Update message contains information about the changed network, such as the network's IP address, the type of change (e.g., addition or withdrawal), and the route's metric (i.e., the cost of the path).

BGP Upda

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: [INST]Explain the different BGP neighbor states and the transitions between them.[/INST]  Border Gateway Protocol (BGP) is a protocol used to exchange routing information between autonomous systems (AS) on the internet. BGP maintains a stateful connection between neighboring routers, and these states are used to determine the best path for forwarding traffic. Here are the different BGP neighbor states and the transitions between them:

1. Idle: This is the initial state when a BGP session is established. The routers exchange the BGP opening message and start the BGP session.
2. Connect: When both routers have exchanged their opening messages, they enter the Connect state. In this state, the routers exchange their routing tables and establish a peer relationship.
3. Active: When one router receives a better route from its peer, it enters the Active state. In this state, the router sends a notification to its peer with the new route. The peer then updates its routing table and en

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Output: [INST]What are the steps for diagnosing BGP route problems, and how can these steps help in troubleshooting BGP issues?[/INST]  Diagnosing and troubleshooting BGP (Border Gateway Protocol) route problems can be a complex process, but there are several steps that can help you identify and resolve the issues. Here are some steps to follow:

1. Gather information: Start by gathering information about the BGP sessions, peers, and routes. You can use the show command in the router to display information about BGP sessions, peers, and routes. For example, you can use the following commands to gather information about BGP sessions:

show ip bgp
show ip bgp peer

2. Check BGP session status: Check the status of BGP sessions to ensure that they are established and functioning correctly. Look for any errors or warnings that may indicate issues with the BGP session.

3. Check peer relationships: Check the relationships between BGP peers to ensure that they are properly configured and func

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: [INST]Can you provide an example of a large-scale service provider BGP implementation and its benefits?[/INST]  Sure, here's an example of a large-scale service provider BGP implementation and its benefits:

Example: Google's BGP Implementation

Google is one of the largest service providers in the world, with a vast network that spans across multiple continents. To manage its global network, Google uses BGP to exchange routing information with its peers and customers. Here's an overview of Google's BGP implementation and its benefits:

1. BGP Peering: Google has established BGP peering relationships with numerous other networks, including ISPs, content delivery networks (CDNs), and cloud providers. This allows Google to exchange routing information with these networks and provide its users with fast and reliable access to online resources.
2. Autonomous System (AS): Google operates its own AS, which is identified by the AS number 15169. This allows Google to manage its own rou

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: [INST]Could you share a notable BGP incident involving route hijacking and its impact on network operations?[/INST]  One notable BGP incident involving route hijacking occurred in 2018, when a misconfigured BGP announcement caused a significant outage for several major internet service providers (ISPs) and online services.

On April 2, 2018, a Pakistani telecom operator, AS45968, accidentally advertised a route for the entire internet to its customers, effectively hijacking all internet traffic destined for any destination. This misconfiguration caused traffic to be diverted through the Pakistani operator's network, rather than the intended path.

The impact of this incident was significant, as many major ISPs and online services, including Comcast, AT&T, and Cloudflare, inadvertently accepted the hijacked routes and passed traffic through the Pakistani operator's network. This resulted in widespread outages and slow internet speeds for users around the world.

The incident was

In [ ]:
import json
import re

input_file_path = "/home/hb/fine-tuning-alpaca/evaluation/bgp_test_multiple_choice.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
ground_truth = [q_dict["answer"] for q_dict in questions_data]

n_question = 0
for q_dict in questions_data:
    question = q_dict["question"]
    options = q_dict["options"]
    
    # Construct prompt with question and options
    prompt = f"{question}\nChoose the correct answer:\n" + "\n".join(options)
    # print(prompt)
    
    # Generate answer using the language model
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Generated Answer: {generated_answer}")
    
        # Parse the generated answer
    answer_lines = generated_answer.splitlines()
    parsed_answer = ""
    for line in answer_lines:
        if line.startswith("Answer: "):
            parsed_answer = re.search(r'\b(\w)\)', line).group(1)
            print(f"Parsed_answer: {parsed_answer}")
            break
    generated_answers.append(parsed_answer)
    n_question += 1
    print(f"----------------{n_question}-----------------")
print(generated_answers)


total_questions = len(questions_data)
correct_answers = 0

for ground_answer, generated_answer in zip(ground_truth, generated_answers):
    if generated_answer.lower() == ground_answer.lower():
        print("Correct")
        correct_answers += 1
    else:
        print("Incorrect")
        print(f"GPT answer: {generated_answer}. Correct answer: {ground_answer}")

accuracy = (correct_answers / total_questions) * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Correct answers: {correct_answers}")
print(f"Incorrect answers: {total_questions - correct_answers}")

In [11]:
import json
import re

input_file_path = "/home/hb/LLM-research/evaluation/bgp_test_multiple_choice_v2.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
ground_truth = [q_dict["answer"] for q_dict in questions_data]

n_question = 0
for q_dict in questions_data:
    question = q_dict["question"]
    options = q_dict["options"]
    
    # Construct prompt with question and options
    prompt = f"{question}\nChoose the correct answer:\n" + "\n".join(options)
    # print(prompt)
    
    # Generate answer using the language model
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Generated Answer: {generated_answer}")
    
        # Parse the generated answer
    answer_lines = generated_answer.splitlines()
    parsed_answer = ""
    for line in answer_lines:
        if line.startswith("Answer: "):
            parsed_answer = re.search(r'\b(\w)\)', line).group(1)
            print(f"Parsed_answer: {parsed_answer}")
            break
    generated_answers.append(parsed_answer)
    n_question += 1
    print(f"----------------{n_question}-----------------")
print(generated_answers)


total_questions = len(questions_data)
correct_answers = 0

for ground_answer, generated_answer in zip(ground_truth, generated_answers):
    if generated_answer.lower() == ground_answer.lower():
        print("Correct")
        correct_answers += 1
    else:
        print("Incorrect")
        print(f"GPT answer: {generated_answer}. Correct answer: {ground_answer}")

accuracy = (correct_answers / total_questions) * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Correct answers: {correct_answers}")
print(f"Incorrect answers: {total_questions - correct_answers}")

Generated Answer: When analyzing BGP route propagation, which attribute provides information about the path that the BGP update has traversed?
Choose the correct answer:
a) AS-Path
b) MED (Multi-Exit Discriminator)
c) Local Preference
d) Next Hop

Answer: a) AS-Path

Explanation:

The AS-Path attribute in BGP updates contains a list of autonomous systems (ASes) that the update has traversed. This attribute provides information about the path that the BGP update has taken from the originating AS to the receiving
Parsed_answer: a
----------------1-----------------
Generated Answer: Which of the following is a primary purpose of BGP?
Choose the correct answer:
a) Establishing communication between devices within a LAN
b) Exchanging routing information between autonomous systems
c) Optimizing local network traffic
d) Assigning IP addresses to devices

Answer: b) Exchanging routing information between autonomous systems

Explanation:

BGP (Border Gateway Protocol) is a routing protocol used

In [3]:
import json
import re

input_file_path = "/home/hb/fine-tuning-alpaca/evaluation/bgp_test_fill_the_blank.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
filled_answers = []

n_question = 0
for q_dict in questions_data:
    question = q_dict["question"]
    answer = q_dict["answer"]

    # Generate answer using the language model
    prompt = f"[INST]{question} \n Fill the blank: \n [/INST]"
    generated_answer = pipe(prompt)[0]['generated_text']
    # print(f"Output: {generated_answer}")
    
    generated_answer = re.search(r'\[/INST\]\s+(.*)', generated_answer, re.DOTALL).group(1).strip()
    if "Sure" in generated_answer:
        generated_answer = generated_answer.split('\n', 1)[1]  # Skip the first line
    print(f"Generated Answer: {generated_answer}")
    generated_answers.append(generated_answer)
    
    # Find the position of the placeholder in the first string
    placeholder_position = question.find("________")

    # Extract the content filled in the placeholder
    filled_content = generated_answer[placeholder_position:placeholder_position + len("autonomous systems (ASes)")]
    filled_answers.append(filled_content)
    n_question += 1
    print(f"Filled content: {filled_content}")
    print(f"---------------------{n_question}-----------------------------")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: 
BGP (Border Gateway Protocol) is a standardized exterior gateway protocol used to exchange routing and reachability information between Autonomous Systems (AS) on the Internet. It enables the selection of paths and routes for data to flow efficiently between networks.
Filled content:  Autonomous Systems (AS) 
---------------------1-----------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: 
In BGP, the AS-Path attribute represents the sequence of autonomous systems (ASes) that a route has traversed.

So, the correct answer is:

"sequence of autonomous systems (ASes)"
Filled content:  sequence of autonomous s
---------------------2-----------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: BGP route selection ______________ involves manipulating the BGP attributes to control the selection of routes.

The correct answer is: BGP route selection ______________ involves manipulating the BGP attributes to control the selection of routes.

Explanation:

BGP (Border Gateway Protocol) is a routing protocol used to exchange routing information among autonomous systems (AS) on the internet. BGP route
Filled content: selection ______________ 
---------------------33-----------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: 
BGP peering is used for establishing BGP session connections and exchanging routing information.
Filled content:  peering is used for esta
---------------------34-----------------------------
Generated Answer: The phenomenon where the Multi-Exit Discriminator (MED) value of a route frequently changes is referred to as "MED oscillations."

Therefore, the correct answer is:

MED oscillations.
Filled content:  as "MED oscillations."


---------------------35-----------------------------


: 